In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
users=pd.read_csv(r'new_dataset/users.csv')
ratings=pd.read_csv(r'new_dataset/ratings.csv')
books=pd.read_csv(r'new_dataset/books.csv')
df=pd.read_csv(r'new_dataset/merge.csv')
df1=pd.read_csv(r'new_dataset/has_ratings.csv')
df2=pd.read_csv(r'new_dataset/no_ratings.csv')

# Top 10 popular books  

In [3]:
data = pd.DataFrame(df1.groupby('ISBN')['Book-Rating'].count()).sort_values('Book-Rating', ascending=False).head(10)
result = pd.merge(data, books, on='ISBN')

In [4]:
result=result[['ISBN','Book-Rating','Book-Title','Book-Author','Year-Of-Publication','Publisher']]

# Yearly popular books

In [5]:
data = pd.DataFrame(df1.groupby('ISBN')['Book-Rating'].count()).sort_values('Book-Rating', ascending=False)
result = pd.merge(data, books, on='ISBN')
result=result[['ISBN','Book-Rating','Book-Title','Book-Author','Year-Of-Publication','Publisher']]
years = set()
indices = []
for ind, row in result.iterrows():
    if row['Year-Of-Publication'] in years:
        indices.append(ind)
    else:
        years.add(row['Year-Of-Publication'])

new_data = result.drop(indices)
new_data = new_data.drop('Book-Rating', axis = 1)
new_data = new_data.sort_values('Year-Of-Publication')

pd.set_option("display.max_rows", None, "display.max_columns", None)
new_data=new_data[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]
new_data

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
92374,1551103982,The Cycling Adventures of Coconut Head: A Nort...,Ted Schredd,1900,Graphic Arts Center Pub Co
125810,0671397214,JOY OF MUSIC P,Leonard Bernstein,1901,Fireside
48039,0373226888,Tommy's Mom,Linda O. Johnston,1902,Harlequin
104633,038528120X,CATCH 22,JOSEPH HELLER,1904,Delta
101127,0404089119,Charlotte Bronte and Her Sisters,Clement K. Shorter,1906,Ams Pr
24715,0911662251,Kybalion: A Study of the Hermetic Philosophy o...,Three Initiates,1908,Yoga Publication Society
58461,0877850836,The Spiritual Life/The Word of God,Emanuel Swedenborg,1911,Steiner Books
5254,0684717603,The GREAT GATSBY (Scribner Classic),F. Scott Fritzgerald,1920,Scribner Paper Fiction
21063,0440772095,RAMONA THE PEST (Ramona Quimby (Paperback)),BEVERLY CLEARY,1923,Yearling
137800,0486600815,Principle of Relativity (Dover Books on Physics),Albert Einstein,1924,Dover Publications


# NEARest neighbour(cosine similarity)

In [19]:
bookName="Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))"
number=10

In [20]:
from scipy.sparse import csr_matrix

popularity_threshold=50

data = (df1.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().
        rename(columns = {'Book-Rating': 'Total-Rating'})[['Book-Title', 'Total-Rating']])

result = pd.merge(data, df1, on='Book-Title')
result = result[result['Total-Rating'] >= popularity_threshold]
result = result.reset_index(drop = True)

matrix = result.pivot_table(index = 'Book-Title', columns = 'User-ID', values = 'Book-Rating').fillna(0)
up_matrix = csr_matrix(matrix)

In [21]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(up_matrix)

distances, indices = model.kneighbors(matrix.loc[bookName].values.reshape(1, -1), n_neighbors = number+1)
print("\nRecommended books:\n")
for i in range(0, len(distances.flatten())):
    if i > 0:
        print(matrix.index[indices.flatten()[i]]) 


Recommended books:

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
The Fellowship of the Ring (The Lord of the Rings, Part 1)
Like Water for Chocolate: A Novel in Monthly Installments With Recipes, Romances and Home Remedies
Anne of Green Gables (Anne of Green Gables Novels (Paperback))
The Client
A Walk to Remember
A Time to Kill
